## Homework6
Chapter6の講義資料などで扱った購買データを利用して以下の問いに答えて、課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
Chapter6の講義資料の総合問題6-1で求めた最も支持度が高かった商品の2つの組み合わせについて、**リフト値**を求めてください。  
最も支持度が高かった商品の`"StockCode"`を、再度求める必要は無く、コード内で直接記入してください。  
<b>`"StockCode"`のデータは事前に文字列に変換してあります．</b>



提出するときは、以下の点に注意してください。  
>- 以下の関数`homework`の`!!WRITE ME!!`に処理を書いてください。(**「`!!WRITE ME!!`」は消して、記入してください。**)
>- 実際の提出は記述された`homework`関数全てになり、**提出はOmnicampus内の宿題の欄から今週の課題を選択後、提出内容に関数を貼り付けてから[Pythonコード提出]を押してください。**
>- `#common`のセルで事前に前処理されたデータ`target_online_retail_data_tb`をそのまま用いてください。
>- 返り値が数値型になるようにしてください。  

In [1]:
# common
import pandas as pd
import numpy as np

## init part(データの読み込みと前処理)
file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])
online_retail_data_table["StockCode"] = online_retail_data_table["StockCode"].astype(str) 

# 数字があるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') 
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)

In [2]:
# レコード数が1,000より多い商品（StockCode）を対象とする
stocknum = target_online_retail_data_tb['StockCode'].value_counts().reset_index()
stocknum = stocknum[stocknum['StockCode'] >= 1000]
stocknum = stocknum.set_index('index')
popgoods = list(stocknum.index)

# すべてのInvoiceNoをtrans_allとして抽出
trans_all = set(target_online_retail_data_tb["InvoiceNo"])

# 支持度が最大になる組合せを調べる
import itertools # itertools

max_support = 0
for i,j in itertools.combinations(popgoods, 2):
    trans_i = set(target_online_retail_data_tb[target_online_retail_data_tb['StockCode'] == i]["InvoiceNo"])
    trans_j = set(target_online_retail_data_tb[target_online_retail_data_tb['StockCode'] == j]["InvoiceNo"])
    trans_ij = trans_i&trans_j
    
    if len(trans_ij)/len(trans_all) > max_support:
        max_support = len(trans_ij)/len(trans_all)
        print(i, j, max_support)

85123A 22423 0.01267803193785067
85123A 85099B 0.013595166163141994
85123A 84879 0.014620198532585239
85123A 47566 0.015051791109192922
85099B 20725 0.022928355632283126
85099B 23203 0.025248165731549418
20725 20727 0.02789167026327147
20725 22383 0.027999568407423393


In [3]:
# working place. everything 
def homework(target_online_retail_data_tb):
    trans_all = set(target_online_retail_data_tb['InvoiceNo'])
    trans_c = set(target_online_retail_data_tb[target_online_retail_data_tb['StockCode']=="20725"]['InvoiceNo'])#商品20725を購入したデータをtrans_Cとする
    trans_d=set(target_online_retail_data_tb[target_online_retail_data_tb['StockCode']=="22383"]['InvoiceNo'])#商品22383を購入したデータをtransdとする
    trans_cd = trans_c&trans_d#商品cdを購入したデータをtrans_cdとする
    support_d = len(trans_d) /len(trans_all)# 全体のバスケットに占める商品の購買率を計算
    confidence = len(trans_cd)/len(trans_c)# 商品Cを購買したときの商品Dの購買率を計算
    lift = confidence / support_d#リフト値を計算
    return lift

**謝辞**：以下のデータセットの利用に関して  
 https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

引用元：Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [[http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml)]. Irvine, CA: University of California, School of Information and Computer Science.
 
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).